In [2]:
!pip install scikit-surprise

     |████████████████████████████████| 11.8 MB 3.7 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1619411 sha256=b168121c11ef28b12f236ffcfcd0f61aa7aa36fdda904d42c5c19ab4375c97a2
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [3]:
from surprise import SVD
from surprise import Dataset
from surprise import accuracy
from surprise.model_selection import train_test_split

In [4]:
data = Dataset.load_builtin("ml-100k")
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


In [5]:
algo = SVD()
algo.fit(trainset)

In [6]:
predictions = algo.test(testset)
print(type(predictions), len(predictions))
predictions[:5]

<class 'list'> 25000


[Prediction(uid='120', iid='282', r_ui=4.0, est=3.6108206170665813, details={'was_impossible': False}),
 Prediction(uid='882', iid='291', r_ui=4.0, est=3.7232971480455435, details={'was_impossible': False}),
 Prediction(uid='535', iid='507', r_ui=5.0, est=4.135233428828235, details={'was_impossible': False}),
 Prediction(uid='697', iid='244', r_ui=5.0, est=3.774850743663557, details={'was_impossible': False}),
 Prediction(uid='751', iid='385', r_ui=4.0, est=3.436918342767461, details={'was_impossible': False})]

In [9]:
[(pred.uid, pred.iid, pred.est) for pred in predictions][:3]

[('120', '282', 3.6108206170665813),
 ('882', '291', 3.7232971480455435),
 ('535', '507', 4.135233428828235)]

In [10]:
uid = str(196)
iid = str(302)
pred = algo.predict(uid, iid)
print(pred)

user: 196        item: 302        r_ui = None   est = 4.43   {'was_impossible': False}


In [27]:
uid = 450; iid = 2
dd = [pred for pred in predictions if pred.uid == str(uid) and pred.iid == str(iid)]
print(len(dd))

0


In [29]:
algo.predict(str(uid), str(iid))

Prediction(uid='450', iid='2', r_ui=None, est=3.5867519232277054, details={'was_impossible': False})

In [30]:
import pandas as pd
ratings = pd.read_csv("/content/drive/MyDrive/ml-data/ml-latest-small/ratings.csv")
ratings.to_csv("/content/drive/MyDrive/ml-data/ml-latest-small/ratings_noh.csv", index=False, header=False)

In [34]:
from surprise import Reader

reader = Reader(line_format="user item rating timestamp", sep=",", rating_scale=(0.5, 5.))
data = Dataset.load_from_file("/content/drive/MyDrive/ml-data/ml-latest-small/ratings_noh.csv", reader=reader)

In [35]:
trainset, testset = train_test_split(data, test_size=.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)

algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [36]:
import pandas as pd
from surprise import Reader, Dataset

ratings = pd.read_csv("/content/drive/MyDrive/ml-data/ml-latest-small/ratings.csv")
reader = Reader(rating_scale=(0.5, 5.0))

data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)
trainset, testset = train_test_split(data, test_size=0.25, random_state=0)

algo = SVD(n_factors=50, random_state=0)
algo.fit(trainset)
predictions = algo.test(testset)
accuracy.rmse(predictions)

RMSE: 0.8682


0.8681952927143516

In [37]:
from surprise.model_selection import cross_validate

cross_validate(algo, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)

Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    0.8726  0.8703  0.8661  0.8722  0.8678  0.8698  0.0025  
MAE (testset)     0.6718  0.6674  0.6656  0.6685  0.6659  0.6678  0.0022  
Fit time          3.30    3.21    3.22    3.19    3.23    3.23    0.04    
Test time         0.29    0.15    0.29    0.16    0.32    0.24    0.07    


{'fit_time': (3.301088809967041,
  3.2147114276885986,
  3.2197062969207764,
  3.189624309539795,
  3.2323544025421143),
 'test_mae': array([0.6717763 , 0.66737459, 0.6655707 , 0.66852063, 0.6659223 ]),
 'test_rmse': array([0.87259358, 0.87031207, 0.86607406, 0.87222473, 0.86778039]),
 'test_time': (0.2921268939971924,
  0.14750885963439941,
  0.29374217987060547,
  0.16342902183532715,
  0.3244161605834961)}

In [38]:
from surprise.model_selection import GridSearchCV
param_grid = {'n_epochs':[20, 40, 60,], 'n_factors':[50, 100, 200]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(data)

print(gs.best_score['rmse'])
print(gs.best_params['rmse'])

0.8764103489178207
{'n_epochs': 20, 'n_factors': 50}


In [96]:
estimator = gs.best_estimator["rmse"]

In [118]:
from surprise.dataset import DatasetAutoFolds

reader = Reader(line_format="user item rating timestamp", sep=",", rating_scale=(0.5, 5.))
data_folds = DatasetAutoFolds(ratings_file="/content/drive/MyDrive/ml-data/ml-latest-small/ratings_noh.csv", reader=reader)
trainset = data_folds.build_full_trainset()
algo = SVD(n_epochs=20, n_factors=50, random_state=0)
algo.fit(trainset)

In [119]:
target_id = 9 

In [120]:
movies = pd.read_csv("/content/drive/MyDrive/ml-data/ml-latest-small/movies.csv")
seen_movies = ratings[ratings['userId'] == target_id]["movieId"].tolist()
total_movies = movies["movieId"].tolist()

In [121]:
unseen_movies = [movie for movie in total_movies if movie not in seen_movies]

In [122]:
predictions = [algo.predict(str(target_id), str(movie_id)) for movie_id in unseen_movies]

In [123]:
def sortkey_est(pred):
  return pred.est

predctions = predictions.sort(key=sortkey_est, reverse=True)

In [124]:
top_predictions = predictions[:10]

In [125]:
predictions

[Prediction(uid='9', iid='858', r_ui=None, est=4.306302135700814, details={'was_impossible': False}),
 Prediction(uid='9', iid='260', r_ui=None, est=4.281663842987387, details={'was_impossible': False}),
 Prediction(uid='9', iid='296', r_ui=None, est=4.278152632122759, details={'was_impossible': False}),
 Prediction(uid='9', iid='1196', r_ui=None, est=4.226073566460876, details={'was_impossible': False}),
 Prediction(uid='9', iid='50', r_ui=None, est=4.1918097904381995, details={'was_impossible': False}),
 Prediction(uid='9', iid='1104', r_ui=None, est=4.154746591122658, details={'was_impossible': False}),
 Prediction(uid='9', iid='1210', r_ui=None, est=4.122016128534504, details={'was_impossible': False}),
 Prediction(uid='9', iid='1213', r_ui=None, est=4.108009609093436, details={'was_impossible': False}),
 Prediction(uid='9', iid='1242', r_ui=None, est=4.083464936588478, details={'was_impossible': False}),
 Prediction(uid='9', iid='593', r_ui=None, est=4.07887165526957, details={'wa

In [126]:
top_movies_id = [ int(pred.iid) for pred in top_predictions]
top_movies_rating = [ pred.est for pred in top_predictions]
top_movies_title = movies.iloc[top_movies_id, :]['title']
print(top_movies_title.tolist())
print(top_movies_rating)

['Escape from New York (1981)', 'Quiz Show (1994)', 'Virtuosity (1995)', 'Picture Perfect (1997)', 'Georgia (1995)', 'Cement Garden, The (1993)', '187 (One Eight Seven) (1997)', 'Kiss Me, Guido (1997)', 'Year of the Horse (1997)', 'Cemetery Man (Dellamorte Dellamore) (1994)']
[4.306302135700814, 4.281663842987387, 4.278152632122759, 4.226073566460876, 4.1918097904381995, 4.154746591122658, 4.122016128534504, 4.108009609093436, 4.083464936588478, 4.07887165526957]


In [127]:
for title, rating in zip(top_movies_title, top_movies_rating):
  print(title, rating)

Escape from New York (1981) 4.306302135700814
Quiz Show (1994) 4.281663842987387
Virtuosity (1995) 4.278152632122759
Picture Perfect (1997) 4.226073566460876
Georgia (1995) 4.1918097904381995
Cement Garden, The (1993) 4.154746591122658
187 (One Eight Seven) (1997) 4.122016128534504
Kiss Me, Guido (1997) 4.108009609093436
Year of the Horse (1997) 4.083464936588478
Cemetery Man (Dellamorte Dellamore) (1994) 4.07887165526957
